In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

# RODAR SOMENTE LOCAL

## Esses testes deletam todos os QueuedEmail e QueuedPush do banco de dados

In [ ]:
!make use-local

In [ ]:
!cat .env

In [ ]:
def print_report():
    !make use-local
    
    push_users = [item.get_full_name() for item in User.objects.filter(user_push_messages__in=QueuedPush.objects.all()).distinct()]
    email_users = [item.get_full_name() for item in User.objects.filter(queuedemail__in=QueuedEmail.objects.all()).distinct()]
    
    print("\nQuantidade de emails criados: {}".format(QueuedEmail.objects.all().count()))
    print("\nQuantidade de push criados: {}".format(QueuedPush.objects.all().count()))
    print("\nUsuarios email ({}): {}".format(len(email_users), sorted(email_users)))
    print("\nUsuarios push ({}): {}".format(len(push_users), sorted(push_users)))
    print("\nTitulos dos emails:\n\n {}".format(
        "\n".join(QueuedEmail.objects.all().values_list("title", flat=True))
    ))

# Descrição e testes das notificações do Sistema

## REGISTROS

### 1) Registro - criação

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `occurrence_record_create`.


- **Enviado para (antigo)**: criador do registro, responsavel pelo registro, quem for adicionado como “Notificado”, usuarios da equipe (se a equipe tiver a flag `sent_to_all_in_firm`).


- **Enviado para (atual)**: quem for adicionado como “Notificado”.


- **Enviado quando**: na criação de um novo registro.


- **Mensagem**: “Um novo registro foi criado”.


### Obs) Como testar no sistema

Ao criar um Registro em uma company de energia, adicione Notificados. Depois da criação do Registro, verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.


In [ ]:
from apps.occurrence_records.notifications import occurrence_record_create
    
occurrence_record = OccurrenceRecord.objects.filter(
    company__name__icontains="uhe",
    occurrencerecord_watchers__isnull=False,
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    occurrence_record_create(occurrence_record)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 2) Registro - edição

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `occurrence_record_update`.


- **Enviado para (antigo)**: criador do registro, responsavel pelo registro, quem for adicionado como “Notificado”, usuarios da equipe (se a equipe tiver a flag `sent_to_all_in_firm`), responsavel da `dest_firm` (se action possuir `dest_firm`).


- **Enviado para (atual)**: criador do registro (se não foi ele quem editou o registro), quem estiver adicionado como “Notificado”.


- **Enviado quando**: na edição de registros.


- **Mensagem**: “Registro UHIT-RG-2020.0001 atualizado”.


### Obs) Como testar no sistema

Edite um Registro em uma company de energia. Depois da edição do Registro, verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.


In [ ]:
from apps.occurrence_records.notifications import occurrence_record_update
    
occurrence_record = OccurrenceRecord.objects.filter(
    company__name__icontains="uhe",
    occurrencerecord_watchers__isnull=False,
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    occurrence_record_update(occurrence_record, True)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 3) Registro - adição aos “notificados”

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `watcher_email_occurrence_record`.


- **Enviado para (antigo)**: quem for adicionado como “Notificado”.


- **Enviado para (atual)**: quem for adicionado como “Notificado”.


- **Enviado quando**: no momento que alguém for adicionado como "Notificado".


- **Mensagem**: "Você será notificado sobre o registro UHIT-RG-2020.0001”.

### Obs) Como testar no sistema

Crie um Registro em uma company de energia. Depois da criação do Registro, adicione Notificados. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.occurrence_records.notifications import occurrence_record_watcher
    
watcher = OccurrenceRecordWatcher.objects.filter(
    occurrence_record__company__name__icontains="uhe",
    user__isnull=False,
    status_email=True,
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    occurrence_record_watcher(watcher)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 4) Registro - inclusão em serviço

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `email_service_order_added`.


- **Enviado para (antigo)**: criador do registro, responsavel pelo registro, quem estiver adicionado como “Notificado”.


- **Enviado para (atual)**: criador do registro, quem estiver adicionado como “Notificado”.


- **Enviado quando**: no momento que um registro é incluso em um Serviço.


- **Mensagem**: “Serviço vinculado ao registro UHIT-RG-2020.0001”.

### Obs) Como testar no sistema

Crie um Registro em uma company de energia. Depois da criação do Registro, adicione esse Registro a um serviço. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.occurrence_records.notifications import occurrence_record_so
    
occurrence_record = OccurrenceRecord.objects.filter(
    company__name__icontains="uhe",
    occurrencerecord_watchers__isnull=False,
    service_orders__isnull=False
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    occurrence_record_so(occurrence_record)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 5) Registro - alteração de aprovação

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `email_status_changed`.


- **Enviado para (antigo)**: responsáveis pela aprovação, equipes responsáveis pela aprovação, usuarios da approver_firm da entidade.


- **Enviado para (atual)**: responsáveis pela aprovação, equipes responsáveis pela aprovação, usuarios da approver_firm da entidade.


- **Enviado quando**: no momento que o status de aprovação de um registro é alterado.


- **Mensagem**: “Registro UHIT-RG-2020.0001 alterado para Necessita Revisão”.

### Obs) Como testar no sistema

Crie um Registro em uma company de energia. Depois da criação do Registro, avance no processo de aprovação desse Registro. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.occurrence_records.notifications import occurrence_record_approval
    
occurrence_record = OccurrenceRecord.objects.filter(
    company__name__icontains="uhe",
    occurrencerecord_watchers__isnull=False,
    approval_step__isnull=False
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    occurrence_record_approval(occurrence_record)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

## **SERVIÇOS E TAREFAS** 

### 1) Tarefa - criação

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `email_procedure_info`.


- **Enviado para (antigo)**: criador da tarefa, responsavel da tarefa, usuarios da equipe (se a equipe tiver a flag `sent_to_all_in_firm`). 


- **Enviado para (atual)**: responsável pela tarefa.


- **Enviado quando**: no momento da criação de uma nova tarefa.


- **Mensagem**: "Nova tarefa: XXXXXX".


### Obs) Como testar no sistema

Dentro de Entregas de alguma Ordem de Serviço em uma company de energia, crie uma nova tarefa. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.


In [ ]:
from apps.service_orders.notifications import email_procedure_info

procedure = Procedure.objects.filter(
    action__service_order__company__name__icontains="uhe",
    responsible__isnull=False
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    email_procedure_info("Procedure", procedure, True)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 2) Serviço - adição aos “Notificados”

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `watcher_email_service_order`.


- **Enviado para (antigo)**: quem for adicionado como “Notificado”.


- **Enviado para (atual)**: quem for adicionado como “Notificado”.


- **Enviado quando**: no momento que alguém for adicionado como "Notificado". 


- **Mensagem**: "Você será notificado sobre o Serviço UHIT-SV-2020.0001”.

### Obs) Como testar no sistema

Dentro de alguma Ordem de Serviço em uma company de energia, adicione Notificados. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.service_orders.notifications import watcher_email_service_order

watcher = ServiceOrderWatcher.objects.filter(
    service_order__company__name__icontains="uhe",
    status_email=True
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    watcher_email_service_order("ServiceOrderWatcher", watcher, True)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 3) Relatório de serviços

- **Tipo**: e-mail.


- **Verticais**: energia.


- **Nome da função**: `service_order_report_email`.


- **Enviado para (antigo)**: quem for adicionado como “Notificado" em serviços.


- **Enviado para (atual)**: quem for adicionado como “Notificado" em serviços.


- **Enviado quando**: de acordo com a frequência selecionada ao adicionar o “Notificado".


- **Mensagem**: "Relatório dos Serviços do dia 27/07 do HidrOS - UHE-ITA“. 

### Obs) Como testar no sistema

Essa notificação é enviada periodicamente.

In [ ]:
from apps.email_handler.asynchronous import service_order_report_email

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    service_order_report_email()
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")


### 4) Relatório gerencial mensal

- **Tipo**: e-mail.


- **Verticais**: energia.


- **Nome da função**: `manager_report_email`.


- **Enviado para (antigo)**: membros de equipes gerenciais em ativos específicos.


- **Enviado para (atual)**: membros de equipes gerenciais em ativos específicos.


- **Enviado quando**: primeiro dia de cada mês.


- **Mensagem**: “Relatório Gerencial de 07/20 do HidrOS - UHE-ITA”.

### Obs) Como testar no sistema

Essa notificação é enviada periodicamente.

In [ ]:
from apps.email_handler.asynchronous import manager_report_email

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    manager_report_email()
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")


## **OBJETOS E BOLETINS DE MEDIÇÃO** 

### 1) Boletim de medição - aprovado

- **Tipo**: e-mail e push.


- **Verticais**: energia.


- **Nome da função**: `approved_measurement_bulletin`.


- **Enviado para (antigo)**: responsaveis da aprovacao, usuarios que modificaram o boletim.


- **Enviado para (atual)**: equipe de Suprimentos, usuário interno que aprovou, usuário externo que aprovou.


- **Enviado quando**: no momento que um Boletim de Medição receber uma aprovação.


- **Mensagem**: "Um novo boletim de medição foi aprovado no objeto XXXXXX".

### Obs) Como testar no sistema

Dentro de algum Boletim em uma company de energia que possua status para aprovar: Aprove esse boletim. Pode ser que as notificações nao sejam criadas se o ApprovalTransition nao tiver a chave `send_notification` dentro do seu callback. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.service_orders.notifications import approved_measurement_bulletin
    
bulletin = MeasurementBulletin.objects.filter(
    contract__isnull=False,
    contract__firm__company__name__icontains="uhe",
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    approved_measurement_bulletin(bulletin, [], "", "")
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

## **APONTAMENTOS**

### 1) Apontamento - nova mensagem

- **Tipo**: e-mail e push.


- **Verticais**: rodovias.


- **Nome da função**: `reporting_message_created`.


- **Enviado para (antigo)**: qualquer usuário que já tenha enviado mensagens naquele apontamento, qualquer usuário que já tenha sido mencionado em mensagens naquele apontamento, membros de equipes que tenham sido mencionadas em mensagens naquele apontamento, usuarios que leram a mensagem, usuarios que modificaram aquele apontamento.


- **Enviado para (atual)**: qualquer usuário que já tenha enviado mensagens naquele apontamento, qualquer usuário que já tenha sido mencionado em mensagens naquele apontamento, membros de equipes que tenham sido mencionadas em mensagens naquele apontamento.


- **Enviado quando**: no momento que uma nova mensagem é criada.


- **Mensagem**: "Nova mensagem no apontamento VPT-SIN-2020.00001“.

### Obs) Como testar no sistema

Dentro de algum Apontamento em uma company de rodovias, envie uma Mensagem. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.reportings.notifications import reporting_message_created
    
reporting_message = ReportingMessage.objects.latest('created_at')

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    reporting_message_created(reporting_message)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 2) Apontamento - atualizações de aprovação

- **Tipo**: e-mail.


- **Verticais**: rodovias.


- **Nome da função**: `reporting_approval_step_report_email`.


- **Enviado para (antigo)**: qualquer usuário que esteja de alguma forma relacionado a um apontamento (criou ou em algum momento editou) cujo status de aprovação tenha sido alterado.


- **Enviado para (atual)**: qualquer usuário que esteja de alguma forma relacionado a um apontamento (criou ou em algum momento editou) cujo status de aprovação tenha sido alterado.


- **Enviado quando**: toda segunda feira, às 7h30.


- **Mensagem**: "Relatório do Fluxo de Aprovação”.

### Obs) Como testar no sistema

Essa notificação é enviada periodicamente.

In [ ]:
from apps.reportings.notifications import reporting_approval_step_report_email
    
try:
    reportings = Reporting.objects.filter(approval_step__isnull=False)
    reporting = reportings[0]
    
    steps = ApprovalStep.objects.filter(
        approval_flow__company=reporting.company,
        step_reportings__in=reportings
    ).exclude(pk=reporting.approval_step.pk).distinct()
    reporting.approval_step = steps[0]
    reporting.save()
    
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    reporting_approval_step_report_email()
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

## **PROGRAMAÇÕES**

### 1) Programação - atualização em apontamentos


- **Tipo**: e-mail.


- **Verticais**: rodovias.


- **Nome da função**: `reporting_job_email`.


- **Enviado para (antigo)**: criador, responsavel, membros da equipe, adicionado como “visualizador".


- **Enviado para (atual)**: qualquer usuário que esteja de alguma forma relacionado a uma programação (criou, é responsável, líder da equipe responsável (e cuidar pra não repetir), foi adicionado como “visualizador”).


- **Enviado quando**: toda segunda feira, às 7h30.


- **Mensagem**: "Relatório de programações - 20/07 a 27/07”.

### Obs) Como testar no sistema

Essa notificação é enviada periodicamente.

In [ ]:
from apps.reportings.notifications import reporting_job_email
    
try:
    reporting = Reporting.objects.filter(job__isnull=False)[0]
    another_job = Job.objects.filter(company=reporting.company).exclude(pk=reporting.job.pk)[0]
    reporting.job = another_job
    reporting.save()
    
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    reporting_job_email()
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")


### 2) Programação - criação


- **Tipo**: e-mail e push.


- **Verticais**: rodovias.


- **Nome da função**: `create_job_email`.


- **Enviado para (antigo)**: usuário responsável e/ou líder da equipe responsável e visualizadores.


- **Enviado para (atual)**: usuário responsável e/ou líder da equipe responsável e visualizadores.


- **Enviado quando**: na criação de uma nova programação.


- **Mensagem**: "Nova Programação - Lorem Ipsum dor Amert acnfsa (descrição da programação) (12 itens)".

### Obs) Como testar no sistema

Na lista de Apontamentos em uma company de rodovias, selecione apontamentos sem programação e crie uma Programação.  Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.work_plans.notifications import create_job_email

job = Job.objects.first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    create_job_email(job)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")


### 3) Programação - adição de novos apontamentos


- **Tipo**: e-mail e push.


- **Verticais**: rodovias.


- **Nome da função**: `update_job_email`.


- **Enviado para (antigo)**: usuário responsável e/ou líder da equipe responsável e visualizadores.


- **Enviado para (atual)**: usuário responsável e/ou líder da equipe responsável e visualizadores.


- **Enviado quando**: ao adicionar novos apontamentos a uma programação existente. Nao enviar mais de 1 notificação por Programação por hora.


- **Mensagem**: "Programação atualizada - Lorem Ipsum dor Amert acnfsa (descrição da programação) (22%)".

### Obs) Como testar no sistema

Na lista de Apontamentos em uma company de rodovias, selecione apontamentos sem programação e adiciona a uma Programação.  Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.work_plans.notifications import update_job_email

job = Job.objects.filter(
    last_notification_sent_at__isnull=True
).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    update_job_email(job)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")


## **USUÁRIOS E ACESSO**

### 1) Fluxo de aprovação - Aprovação por e-mail

- **Tipo**: e-mail e push.


- **Verticais**: rodovias e energia.


- **Nome da função**: `send_approval_step_email`.


- **Enviado para (antigo)**: pessoas que podem aprovar aquela determinada solicitação de acesso.


- **Enviado para (atual)**: pessoas que podem aprovar aquela determinada solicitação de acesso.


- **Enviado quando**: uma solicitação de acesso é criada, ou o passo de aprovação é alterado.


- **Mensagem**: "Solicitação de Acesso - Marcos Hollerweger - UHE-ITA“.

### Obs) Como testar no sistema

Crie uma Solicitação de acesso. As notificações podem nao serem criadas se nao exister a flag `send_approval_step_email` no metadata da company. Depois verifique no djang-admin a criação dos respectivos objetos QueuedEmail e QueuedPush.

In [ ]:
from apps.companies.notifications import send_approval_step_email

try:
    access_request = AccessRequest.objects.filter(
        done=False, 
        approval_step__approval_flow__company__metadata__send_approval_step_email=True
    ).order_by('-created_at').first()
    
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    send_approval_step_email(access_request.approval_step, access_request)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 2) Usuário - redefinição de senha

Mudar para apenas plain text e adicionar informação de que precisa ser acessado pelo Chrome ou Firefox.

- **Tipo**: e-mail.


- **Verticais**: energia e rodovias.


- **Nome da função**: `password_reset_token_created`.


- **Enviado para (antigo)**: usuário que solicitar redefinição de senha. 


- **Enviado para (atual)**: usuário que solicitar redefinição de senha.


- **Enviado quando**: após a solicitação de redefinição de senha. 


- **Mensagem**: "Atualização de Senha”.

### Obs) Como testar no sistema

Solicite uma nova senha na pagina de login. Depois verifique no djang-admin a criação do QueuedEmail.

In [ ]:
from django_rest_passwordreset.models import ResetPasswordToken
from apps.users.notifications import password_reset_token_created

try:
    reset_token = ResetPasswordToken.objects.create(user=User.objects.get(username="rlcs"))
    
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    password_reset_token_created("ResetPasswordRequestToken", None, reset_token)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 3) Usuário - senha inicial

Mudar para apenas plain text e adicionar informação de que precisa ser acessado pelo Chrome ou Firefox

- **Tipo**: e-mail.


- **Verticais**: energia e rodovias.


- **Nome da função**: `send_email_password`.


- **Enviado para (antigo)**: usuário que acaba de ser criado e não possui ID de autenticação integrada associado.


- **Enviado para (atual)**: usuário que acaba de ser criado e não possui ID de autenticação integrada associado.


- **Enviado quando**: no momento da criação do usuário.


- **Mensagem**: “Senha da Conta”.

### Obs) Como testar no sistema

Crie um novo usuário com email. Depois verifique no djang-admin a criação do QueuedEmail.

In [ ]:
from apps.users.notifications import send_email_password

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    send_email_password({
        "username": "marcos",
        "password": "asdadsa",
        "email": "marcos@marcosh.eng.br"
    }, User.objects.get(username="rlcs"))
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 4) Registro de usuário


- **Tipo**: e-mail e push.


- **Verticais**: energia e rodovias.


- **Nome da função**: `email_procedure_registration`.


- **Enviado para (antigo)**: usuário que será adicionado no sistema.


- **Enviado para (atual)**: usuário que será adicionado no sistema.


- **Enviado quando**: no momento do registro do usuário.


- **Mensagem**: “Nova tarefa: Aprovação de novo usuário”.

### Obs) Como testar no sistema

No momento não está sendo usado no sistema.

In [ ]:
from apps.users.notifications import email_procedure_registration

try:
    registration = RegistrationUser.objects.first()
    
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    email_procedure_registration("RegistrationUser", registration, True)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

## **OUTROS**

### 1) Exportação - pronta para download

- **Tipo**: e-mail.


- **Verticais**: rodovias.


- **Nome da função**: `send_email_export_request`.


- **Enviado para (antigo)**: usuário que soliciou a exportação.


- **Enviado para (novo)**: usuário que soliciou a exportação.


- **Enviado quando**: no momento que a exportação é finalizada pelo sistema.


- **Mensagem**: "Arquivo zip pronto para download”.

### Obs) Como testar no sistema

Na lista de Apontamentos, selecione alguns Apontamentos e baixe um exportação ZIP. Espere ela terminar (verificando nas configurações). Depois verifique no djang-admin a criação do QueuedEmail.


In [ ]:
from apps.templates.notifications import send_email_export_request

export_request = ExportRequest.objects.filter(done=True).first()

try:
    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    send_email_export_request(export_request)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

### 2) Sincronização - finalizada

- **Tipo**: e-mail e push.


- **Verticais**: rodovias.


- **Nome da função**: `email_mobile_sync`.


- **Enviado para (atualmente)**: conjunto de usuários-chave de uma unidade.


- **Enviado para (novo)**: conjunto de usuários-chave de uma unidade.


- **Enviado quando**: no momento que uma sincronização é dada como concluída pelo aplicativo, ou 30 minutos após uma sincronização ter sido iniciada, caso não seja concluída nesse tempo.  


- **Mensagem**: "Nova sincronização de Pedro Fornari”. 

### Obs) Como testar no sistema

Sincronize seu aplicativo. Depois verifique no djang-admin a criação dos respectivos objetos de QueuedEmail e QueuedPush.

In [ ]:
from apps.templates.notifications import email_mobile_sync

try:
    export_request = ExportRequest.objects.filter(done=True).first()
    reporting = Reporting.objects.filter(company=export_request.company).latest('created_at')

    mobile_sync = MobileSync.objects.create(
        company=reporting.company,
        done=True,
        created_by=reporting.created_by,
        kind="FULL_SYNC",
        email_sent=False,
    )

    hist = reporting.history.latest()

    hist.mobile_sync = mobile_sync
    hist.save()

    mobile_sync.email_sent = False
    mobile_sync.save()

    reporting.company.key_users.set(User.objects.filter(username="rlcs"))

    QueuedEmail.objects.all().delete()
    QueuedPush.objects.all().delete()
    
    email_mobile_sync(mobile_sync)
    
    print_report()
except Exception:
    raise Exception("Dados para o teste não encontrados")

